In [35]:
#!pip install backtrader
#!pip install strategies
#!pip install yfinance

  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8488 sha256=62a5731a29f21e53bc92302b13539fe745279e9a5fd854b4085e782ae0c00bd9
  Stored in directory: c:\users\falda\appdata\local\pip\cache\wheels\21\c9\66\b41c847de65c7985db52ec21d59996841598b8b0e93f2b9500
Successfully built multitasking
  Attempting uninstall: requests
    Found existing installation: requests 2.13.0
    Uninstalling requests-2.13.0:
      Successfully uninstalled requests-2.13.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
cbpro 1.1.4 requires requests==2.13.0, but you have requests 2.26.0 which is incompatible.
azure-core 1.18.0 requires six>=1.11.0, but you have six 1.10.0 which is incompatible.


In [37]:
import backtrader as bt
import matplotlib.pyplot as plt
import yfinance as yf

In [47]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [51]:
data = bt.feeds.YahooFinanceCSVData(dataname='Data/file.csv',reverse=False)

cerebro = bt.Cerebro()

cerebro.adddata(data)

cerebro.addstrategy(TestStrategy)

cerebro.broker.setcommission(commission=0.001)

cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2014-09-17, Close, 457.33
2014-09-18, Close, 424.44
2014-09-18, BUY CREATE, 424.44
2014-09-19, BUY EXECUTED, Price: 424.10, Cost: 424.10, Comm 0.42
2014-09-19, Close, 394.80
2014-09-20, Close, 408.90
2014-09-21, Close, 398.82
2014-09-22, Close, 402.15
2014-09-23, Close, 435.79
2014-09-24, Close, 423.20
2014-09-24, SELL CREATE, 423.20
2014-09-25, SELL EXECUTED, Price: 423.16, Cost: 424.10, Comm 0.42
2014-09-25, OPERATION PROFIT, GROSS -0.94, NET -1.79
2014-09-25, Close, 411.57
2014-09-25, BUY CREATE, 411.57
2014-09-26, BUY EXECUTED, Price: 411.43, Cost: 411.43, Comm 0.41
2014-09-26, Close, 404.42
2014-09-27, Close, 399.52
2014-09-28, Close, 377.18
2014-09-29, Close, 375.47
2014-09-30, Close, 386.94
2014-10-01, Close, 383.61
2014-10-01, SELL CREATE, 383.61
2014-10-02, SELL EXECUTED, Price: 383.99, Cost: 411.43, Comm 0.38
2014-10-02, OPERATION PROFIT, GROSS -27.44, NET -28.24
2014-10-02, Close, 375.07
2014-10-02, BUY CREATE, 375.07
2014-10-03, BUY EXECUT

2016-02-28, Close, 433.50
2016-02-29, Close, 437.70
2016-02-29, SELL CREATE, 437.70
2016-03-01, SELL EXECUTED, Price: 437.92, Cost: 420.96, Comm 0.44
2016-03-01, OPERATION PROFIT, GROSS 16.96, NET 16.10
2016-03-01, Close, 435.12
2016-03-02, Close, 423.99
2016-03-02, BUY CREATE, 423.99
2016-03-03, BUY EXECUTED, Price: 423.91, Cost: 423.91, Comm 0.42
2016-03-03, Close, 421.65
2016-03-04, Close, 410.94
2016-03-05, Close, 400.57
2016-03-06, Close, 407.71
2016-03-07, Close, 414.32
2016-03-08, Close, 413.97
2016-03-08, SELL CREATE, 413.97
2016-03-09, SELL EXECUTED, Price: 413.89, Cost: 423.91, Comm 0.41
2016-03-09, OPERATION PROFIT, GROSS -10.02, NET -10.86
2016-03-09, Close, 414.86
2016-03-10, Close, 417.13
2016-03-11, Close, 421.69
2016-03-12, Close, 411.62
2016-03-13, Close, 414.07
2016-03-14, Close, 416.44
2016-03-15, Close, 416.83
2016-03-16, Close, 417.01
2016-03-17, Close, 420.62
2016-03-18, Close, 409.55
2016-03-19, Close, 410.44
2016-03-20, Close, 413.76
2016-03-21, Close, 413.31
20

2017-09-26, Close, 3892.35
2017-09-27, Close, 4200.67
2017-09-28, Close, 4174.73
2017-09-29, Close, 4163.07
2017-09-29, BUY CREATE, 4163.07
2017-09-30, BUY EXECUTED, Price: 4166.11, Cost: 4166.11, Comm 4.17
2017-09-30, Close, 4338.71
2017-10-01, Close, 4403.74
2017-10-02, Close, 4409.32
2017-10-03, Close, 4317.48
2017-10-04, Close, 4229.36
2017-10-05, Close, 4328.41
2017-10-05, SELL CREATE, 4328.41
2017-10-06, SELL EXECUTED, Price: 4324.46, Cost: 4166.11, Comm 4.32
2017-10-06, OPERATION PROFIT, GROSS 158.35, NET 149.86
2017-10-06, Close, 4370.81
2017-10-07, Close, 4426.89
2017-10-08, Close, 4610.48
2017-10-09, Close, 4772.02
2017-10-10, Close, 4781.99
2017-10-11, Close, 4826.48
2017-10-12, Close, 5446.91
2017-10-13, Close, 5647.21
2017-10-14, Close, 5831.79
2017-10-15, Close, 5678.19
2017-10-16, Close, 5725.59
2017-10-17, Close, 5605.51
2017-10-18, Close, 5590.69
2017-10-18, BUY CREATE, 5590.69
2017-10-19, BUY EXECUTED, Price: 5583.74, Cost: 5583.74, Comm 5.58
2017-10-19, Close, 5708.5

2018-12-29, Close, 3820.41
2018-12-30, Close, 3865.95
2018-12-31, Close, 3742.70
2019-01-01, Close, 3843.52
2019-01-02, Close, 3943.41
2019-01-03, Close, 3836.74
2019-01-04, Close, 3857.72
2019-01-05, Close, 3845.19
2019-01-06, Close, 4076.63
2019-01-07, Close, 4025.25
2019-01-08, Close, 4030.85
2019-01-09, Close, 4035.30
2019-01-10, Close, 3678.92
2019-01-11, Close, 3687.37
2019-01-12, Close, 3661.30
2019-01-13, Close, 3552.95
2019-01-13, BUY CREATE, 3552.95
2019-01-14, BUY EXECUTED, Price: 3557.31, Cost: 3557.31, Comm 3.56
2019-01-14, Close, 3706.05
2019-01-15, Close, 3630.68
2019-01-16, Close, 3655.01
2019-01-17, Close, 3678.56
2019-01-18, Close, 3657.84
2019-01-19, Close, 3728.57
2019-01-19, SELL CREATE, 3728.57
2019-01-20, SELL EXECUTED, Price: 3725.45, Cost: 3557.31, Comm 3.73
2019-01-20, OPERATION PROFIT, GROSS 168.14, NET 160.86
2019-01-20, Close, 3601.01
2019-01-21, Close, 3576.03
2019-01-21, BUY CREATE, 3576.03
2019-01-22, BUY EXECUTED, Price: 3575.08, Cost: 3575.08, Comm 3.5

2020-01-01, BUY EXECUTED, Price: 7194.89, Cost: 7194.89, Comm 7.19
2020-01-01, Close, 7200.17
2020-01-02, Close, 6985.47
2020-01-03, Close, 7344.88
2020-01-04, Close, 7410.66
2020-01-05, Close, 7411.32
2020-01-06, Close, 7769.22
2020-01-06, SELL CREATE, 7769.22
2020-01-07, SELL EXECUTED, Price: 7768.68, Cost: 7194.89, Comm 7.77
2020-01-07, OPERATION PROFIT, GROSS 573.79, NET 558.83
2020-01-07, Close, 8163.69
2020-01-08, Close, 8079.86
2020-01-09, Close, 7879.07
2020-01-09, BUY CREATE, 7879.07
2020-01-10, BUY EXECUTED, Price: 7878.31, Cost: 7878.31, Comm 7.88
2020-01-10, Close, 8166.55
2020-01-11, Close, 8037.54
2020-01-12, Close, 8192.49
2020-01-13, Close, 8144.19
2020-01-14, Close, 8827.76
2020-01-15, Close, 8807.01
2020-01-15, SELL CREATE, 8807.01
2020-01-16, SELL EXECUTED, Price: 8812.48, Cost: 7878.31, Comm 8.81
2020-01-16, OPERATION PROFIT, GROSS 934.17, NET 917.48
2020-01-16, Close, 8723.79
2020-01-16, BUY CREATE, 8723.79
2020-01-17, BUY EXECUTED, Price: 8725.21, Cost: 8725.21, C

2021-05-18, Order Canceled/Margin/Rejected
2021-05-18, Close, 42909.40
2021-05-18, BUY CREATE, 42909.40
2021-05-19, Order Canceled/Margin/Rejected
2021-05-19, Close, 37002.44
2021-05-19, BUY CREATE, 37002.44
2021-05-20, Order Canceled/Margin/Rejected
2021-05-20, Close, 40782.74
2021-05-21, Close, 37304.69
2021-05-22, Close, 37536.63
2021-05-23, Close, 34770.58
2021-05-24, Close, 38705.98
2021-05-25, Close, 38402.22
2021-05-26, Close, 39294.20
2021-05-27, Close, 38436.97
2021-05-28, Close, 35697.61
2021-05-28, BUY CREATE, 35697.61
2021-05-29, Order Canceled/Margin/Rejected
2021-05-29, Close, 34616.07
2021-05-29, BUY CREATE, 34616.07
2021-05-30, Order Canceled/Margin/Rejected
2021-05-30, Close, 35678.13
2021-05-31, Close, 37332.86
2021-06-01, Close, 36684.93
2021-06-02, Close, 37575.18
2021-06-03, Close, 39208.77
2021-06-04, Close, 36894.41
2021-06-05, Close, 35551.96
2021-06-05, BUY CREATE, 35551.96
2021-06-06, Order Canceled/Margin/Rejected
2021-06-06, Close, 35862.38
2021-06-07, Close